<a href="https://colab.research.google.com/github/sitori8354/eatago/blob/main/Eatago.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -U pywebio
# !pip install -U https://github.com/solrz/PyWebIO/archive/dev.zip

     |████████████████████████████████| 317kB 6.7MB/s 
  Created wheel for pywebio: filename=pywebio-1.2.3-cp37-none-any.whl size=322618 sha256=805f4acb3d69bf53d99bc3884d6d10e0b9db8fac0e4e2562110989e95a4ddaed
  Stored in directory: /root/.cache/pip/wheels/f6/23/85/82f7b3b2b4bd9c3d171a9b3b1021aad59ec05d638e55954410
Successfully built pywebio


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!wget https://raw.githubusercontent.com/solrz/pywebio-example/main/hello_world.py

!pip3 install nest-asyncio

--2021-05-14 14:43:55--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.203.109.182, 52.202.162.45, 3.221.4.217, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.203.109.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  18.9MB/s    in 0.7s    

2021-05-14 14:43:56 (18.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
--2021-05-14 14:43:57--  https://raw.githubusercontent.com/solrz/pywebio-example/main/hello_world.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, a

In [6]:
import random
import os
import time
assign_port = 80
import nest_asyncio
from pywebio.input import *
from pywebio.output import *
import pywebio
from multiprocessing import Process
import sys, json
import asyncio

try:
  del new_server
  del start_app
except:
  pass

nest_asyncio.apply()
def new_server(target_app, port):
  try:
    if pywebio:
      from importlib import reload
      reload(pywebio)
  except:
    pass
  return lambda: pywebio.start_server(target_app, port=port)

previous_process = None
ports = [80]

def start_app(target_app):
  assign_port = random.randint(5000,9999)
  global ports
  while assign_port in ports:
    assign_port += 1
  ports += [assign_port]
  tunnel_port = f'1{assign_port}'
  ngrok_file = f'./ngrok_config_{tunnel_port}'
  with open(ngrok_file, 'w') as f:
    f.write('web_addr: '+tunnel_port)
  get_ipython().system_raw(f'./ngrok http {assign_port} --config "{ngrok_file}" &')
  time.sleep(2)
  forward_info_raw = !curl -s http://localhost:$tunnel_port/api/tunnels 
  # print(forward_info_raw)

  forward_info = json.loads(forward_info_raw[0])
  print(f'請拖曳網址到新視窗來打開App（每次網址都會更新喔！）: \n{forward_info["tunnels"][0]["public_url"]}')
  # | python3 -c \
  #    'import sys, json; print("請拖曳網址到新視窗來打開App（每次網址都會更新喔！）: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'
  global previous_process
  if previous_process:
    previous_process.terminate()
  previous_process = Process(target=new_server(target_app, assign_port))
  previous_process.daemon = True
  previous_process.start()



In [ ]:
def example_hello_world_app():
  put_text('hello_world！')
  name = input('花枝魷魚麵？')
  put_text(f'{name} 早安, 歡迎使用我的App')

def inertactive_app():
  enable_print_convert_to_put_text = True
  put_markdown('# Python 即時編譯Web App')
  put_markdown('在這裡，你可以直接輸入程式碼，快速測試你的code')
  code = "put_markdown('# 我可以讀透你的內心...')\nname = input('首先，請問大名？')\nput_text(f'你的名字叫{name}，猜對了吧！')\n"
  round = 1
  while True:
    put_text(f'第{round}次執行')
    code = textarea('輸入程式', code={
      'mode': "python",  # code language
      'theme': 'darcula',  # Codemirror theme. Visit https://codemirror.net/demo/theme.html#cobalt to get more themes
    }, value=code)
    put_markdown('---')
    put_code(code)
    try:
      exec(code.replace('print(','put_text('))
    except Exception as e:
      put_text(e)
    put_markdown('---')
    round += 1


# 把 start_app(example_hello_world_app) 的 「example_hello_world_app」是會執行App


請拖曳網址到新視窗來打開App（每次網址都會更新喔！）: 
https://88bcfb5e4a60.ngrok.io
Listen on 0.0.0.0:5801


In [7]:
def edit():
    put_text("You click edit button")
    
def delete():
    put_text("You click delete button")

def action():
  put_buttons(['edit', 'delete'], onclick=[edit, delete])

start_app(action)



請拖曳網址到新視窗來打開App（每次網址都會更新喔！）: 
https://e09c1d6789e2.ngrok.io
Listen on 0.0.0.0:6574
